In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from typing import Optional, Callable
import os
import matplotlib.pyplot as plt

from scripts.loops import train_loop, valid_loop, test_loop
from scripts.dataset_template import DatasetTemplate
from scripts.models.autoencoder_cnn import CNNAutoencoder
from scripts.models.autoencoder_lin import LinearAutoencoder
from scripts.models.autoencoder_lstm import LSTMAutoencoder
import pandas as pd

In [2]:
#check if cuda available

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [3]:
torch.manual_seed(0)
np.random.seed(0)


In [4]:
#paths
path_data_main = "Data_abs_only"

data_train_path = f"/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/{path_data_main}/Data_channels/train/comeretial"
data_test_folder_path = f"/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/{path_data_main}/Data_channels/test"
data_valid_folder_path = f"/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/{path_data_main}/Data_channels/valid"

testing_folders = os.listdir(data_test_folder_path)
valid_folders = os.listdir(data_valid_folder_path)

In [5]:
def save_file(file_name, values):
    with open(file_name, "w") as output:
        output.write(str(values))

In [6]:
#hyperparameters
batch_size = 32
batch_size_test_valid = 1
learning_rate = 0.001
epochs = 200

In [7]:
#transforms
data_transforms = transforms.Compose([transforms.ToTensor()])

In [8]:
saving_path = '/home/bolci/Documents/Projekty/5G_OPEN_RAN/Anomaly_detection/5G_Open_RAN/scripts/Results/'

In [9]:
#datasets and dataloaders


b_s = [16, 32, 64, 128]
learnin_rt = [0.001, 0.0001, 0.00001]

for batch_size in b_s:
    for learning_rate in learnin_rt:

        folder_name = F"results_batch-size={batch_size}_learning-rate={learning_rate}"
        saving_path_with_folder = os.path.join(saving_path, folder_name)
        
        if not os.path.exists(saving_path_with_folder):
            os.makedirs(saving_path_with_folder)

        #train
        train_dataset = DatasetTemplate(data_train_path, 0, transform = data_transforms)
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        #valid
        valid_datasets = {}
        valid_dataloaders = {}
        
        for valid_folder_name in valid_folders:
            valid_folder_path_all = os.path.join(data_valid_folder_path, valid_folder_name)
            valid_datasets[valid_folder_name] = DatasetTemplate(valid_folder_path_all, 1)
            valid_dataloaders[valid_folder_name] = DataLoader(valid_datasets[valid_folder_name], batch_size=batch_size, shuffle=True)
        
        #test
        test_datasets = {}
        test_dataloaders = {}
        
        for test_folder_name in testing_folders:
            test_folder_path_all = os.path.join(data_test_folder_path, test_folder_name)
            test_datasets[test_folder_name] = DatasetTemplate(test_folder_path_all, 1)
            test_dataloaders[test_folder_name] = DataLoader(test_datasets[test_folder_name], batch_size=batch_size_test_valid, shuffle=True)


        #define model
        model = CNNAutoencoder().to(device)
        criterion = nn.MSELoss()
        loss_RMS = lambda x,y: torch.sqrt(criterion(x, y))
        metric_fn_RMS = lambda x,y: torch.sqrt(criterion(x, y))
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        #for x,x in train_dataloader:
        #    print(model(x.to('cuda')).shape)
        
        
        valid_loop(test_dataloaders[list(test_dataloaders.keys())[0]],
                  model,
                  loss_RMS,
                  metric_fn_RMS,
                  device=device)


        #epochs

        # Lists to log the losses during the training process
        train_losses = []
        
        valid_losses_true = []
        valid_metrics_true = []
        
        valid_losses_false = []
        valid_metrics_false = []
        
        for t in range(epochs):
          print(f"Epoch {t+1}\n-------------------------------")
          train_loss = train_loop(train_dataloader,
                                             model,
                                             loss_RMS,
                                             optimizer)
        
          print("------------- TRUE BTS SCORE ----------------")
          valid_loss_true, valid_metric_true = valid_loop(valid_dataloaders['Fake_Bts_PCI_466'],
                                                          model,
                                                          loss_RMS,
                                                          metric_fn_RMS)
        
          print("------------- FALSE BTS SCORE ----------------")
          valid_loss_false, valid_metric_false = valid_loop(valid_dataloaders['Fake_Bts_PCI_466_wPA'],
                                                            model,
                                                            loss_RMS,
                                                            metric_fn_RMS)
            
        
          train_losses.append(train_loss)
        
          valid_losses_true.append(valid_loss_true)
          valid_metrics_true.append(valid_metric_true)
        
          valid_losses_false.append(valid_loss_false)
          valid_metrics_false.append(valid_metric_false)
            
        
        print("Done!")

        model_save = 'model.pt'
        saving_path_model = os.path.join(saving_path_with_folder, model_save)
        torch.save(model.state_dict(), saving_path_model)

        saving_path_figure = os.path.join(saving_path_with_folder, 'fig_1.png')
        plt.figure()
        plt.plot(train_losses[10:], label="train")
        plt.plot(valid_losses_true[10:], label="losses True BTS")
        plt.plot(valid_losses_false[10:], label="losses False BTS")
        plt.legend()
        plt.savefig(saving_path_figure)

        saving_path_figure = os.path.join(saving_path_with_folder, 'fig_2.png')
        plt.figure()
        plt.plot(train_losses, label="train")
        plt.plot(valid_losses_true, label="losses True BTS")
        plt.plot(valid_losses_false, label="losses False BTS")
        plt.legend()
        plt.savefig(saving_path_figure)

        
        #saving scores
        train_saving_path = os.path.join(saving_path_with_folder, 'train_scores.txt')
        save_file(train_saving_path, train_losses)

        validT_saving_path = os.path.join(saving_path_with_folder, 'valid_true_scores.txt')
        save_file(validT_saving_path, valid_losses_true)

        validF_saving_path = os.path.join(saving_path_with_folder, 'valid_false_scores.txt')
        save_file(validF_saving_path, valid_losses_false)

        
        for testing_name, testing_loader in test_dataloaders.items():
            test_losses, test_metrics = test_loop(testing_loader, 
                                                    model,
                                                    loss_RMS,
                                                     metric_fn_RMS)

            #saving_scores
            validF_saving_path = os.path.join(saving_path_with_folder, f'Ftest_{testing_name}_scores.txt')
            save_file(validF_saving_path, test_losses)
            
            mean_loss = np.mean(np.array(test_losses))
            mean_metrics = np.mean(np.array(test_metrics))
            print(f"Dataset name: {testing_name}, mean_loss: {mean_loss}, mean_acc: {mean_metrics}")

        

                

Avg loss: 0.492267 

Epoch 1
-------------------------------
loss: 0.338245  [    0/163420]



KeyboardInterrupt: 

In [ ]:
for k,v in scores.items():
    print(k)
    print(len(v))